In [1]:
import pyautogui
import time
import pyperclip
import pytesseract
from PIL import ImageGrab
import re

In [2]:
import pytesseract

# Replace the path below with the actual path where Tesseract is installed
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [3]:
# 이미지에서 텍스트 추출 및 숫자 반환
def extract_number_from_image(x1, y1, x2, y2):
    screenshot = ImageGrab.grab(bbox=(x1, y1, x2, y2))
    text = pytesseract.image_to_string(screenshot)
    numbers = re.findall(r'\d+', text)
    return int(numbers[0]) if numbers else None

In [4]:
time.sleep(2)
pyautogui.click(350, 70)
time.sleep(2)
pyautogui.moveTo(1304, 66)
pyautogui.dragTo(1304, 895, duration=2.0)
pyautogui.keyDown('shift')
pyautogui.click(158, 902)
pyautogui.keyUp('shift')
time.sleep(2)
pyautogui.click(1395, 215)
time.sleep(2)
number_extracted = extract_number_from_image(656, 436, 870, 470)
print(f"Extracted number: {number_extracted}")


Extracted number: 308
